In [49]:
import pandas as pd
import math
import numpy as np
import librosa
import os
from pydub import AudioSegment
import keras.models
from werkzeug.utils import secure_filename
from tensorflow.keras.models import load_model


In [75]:
def make_prediction(data):
    bird_model = load_model('cnn_66_trained.h5')
    data_array = np.array(data)
    X = data_array[..., np.newaxis]
    prediction_array = bird_model.predict(X)

    predicted_index = np.argmax(prediction_array, axis=1)

    species_df = pd.read_csv('bird_sightings.csv')

    species_list = []
    common_names = []
    for row in species_df:
        if row['Species'] in species_list:
            pass
        else:
            species_list.append(row['Species'])
        if row['English_name'] in common_names:
            pass
        else:
            common_names.append(row['English_name'])

    d = {'Species': species_list, 'Common Name': common_names}
    name_df = pd.DataFrame(d)
    name_df = name_df.sort_values('Species')
    name_df = name_df.reset_index(drop=True)
    
    for index, rows in name_df.iterrows():
        if int(predicted_index) == int(index):
            result = row['Species'], row['Common Name']
        break
        
    return result


In [51]:
# load audio file, transform, extract MFCCs
def transform_file(file):
    sample_rate = 22050
    n_mfcc = 26
    n_fft = 2048
    hop_length = 512
    num_segments = 1
    duration = 30
    samples_per_track = sample_rate * duration 
    num_samples_per_segment = int(samples_per_track / num_segments)
    expected_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)

    signal, sr = librosa.load(file, sr = sample_rate)
    for s in range(num_segments):
        start_sample = num_samples_per_segment * s
        finish_sample = start_sample + num_samples_per_segment
        mfccs = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                    sr = sample_rate, n_fft = n_fft, 
                                    n_mfcc = n_mfcc, hop_length = hop_length)
        mfccs = mfccs.T
        if len(mfccs) == expected_vectors_per_segment: 
            data = np.array(mfccs)
    
    return data


In [52]:
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [53]:
def upload_file(file):
#     file = request.files['file']
#     if file and allowed_file(file.filename):
    name, ext = os.path.splitext(file)
    #for wav files
    if ext == 'wav':
        sound = AudioSegment.from_wav(file)
        # make 30s clip
        if len(sound) < 30000:
            duration = 30000-len(sound)
            segment = AudioSegment.silent(duration = duration)
            extract =  sound + segment
        else:
            extract = sound[0:30000]
    #for mp3 files
    else:
        sound = AudioSegment.from_mp3(file)
        # make 30s clip 
        if len(sound) < 30000:
            duration = 30000-len(sound)
            segment = AudioSegment.silent(duration = duration)
            extract =  sound + segment
        else:
            extract = sound[0:30000]
            
    wav_file = extract.export('data.wav', format = 'wav')
    data = transform_file(wav_file)
        
    prediction = make_prediction(data)

    return prediction

In [54]:
file = '../test_files/XC546020 - Bluethroat - Luscinia svecica.mp3'

# upload_file(file)

In [77]:

sound = AudioSegment.from_mp3(file)
if len(sound) < 30000:
    duration = 30000-len(sound)
    segment = AudioSegment.silent(duration = duration)
    extract =  sound + segment
else:
    extract = sound[0:30000]
    
wav_file = extract.export(f'{name}.wav', format = 'wav')

# sample_rate = 22050
# n_mfcc = 26
# n_fft = 2048
# hop_length = 512
# num_segments = 1
# duration = 30
# samples_per_track = sample_rate * duration 
# num_samples_per_segment = int(samples_per_track / num_segments)
# expected_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)



NameError: name 'name' is not defined

In [76]:
signal, sr = librosa.load(wav_file, sr = sample_rate)

mfcc_list = []

for s in range(num_segments):
    start_sample = num_samples_per_segment * s
    finish_sample = start_sample + num_samples_per_segment
    mfccs = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                    sr = sample_rate, n_fft = n_fft, 
                                    n_mfcc = n_mfcc, hop_length = hop_length)
    mfccs = mfccs.T
    if len(mfccs) == expected_vectors_per_segment: 
        mfcc_list.append(np.array(mfccs))
print(mfcc_list)

RuntimeError: Error opening <_io.BufferedRandom name='data.wav'>: File contains data in an unknown format.

In [57]:
data = np.array(mfcc_list)

data.shape

(1, 1292, 26)

In [58]:
X = data[..., np.newaxis]

X.shape

(1, 1292, 26, 1)

In [59]:
model = load_model('cnn_66_trained.h5')

In [60]:
predict_array = model.predict(X)

print(predict_array)

[[1.69042860e-08 2.15302748e-06 1.37453826e-04 4.95772987e-08
  6.12660301e-07 2.33932269e-07 1.97893701e-06 4.03095646e-06
  8.80501887e-07 4.83149856e-01 4.10694111e-06 3.80469999e-07
  2.82663498e-12 2.46412313e-08 1.79181711e-07 5.90283307e-08
  1.08362365e-04 1.81425122e-07 1.69866416e-06 1.62184715e-01
  1.99661194e-08 2.46889740e-02 1.23162886e-06 2.28287372e-05
  2.85708524e-09 1.69943320e-07 1.38473843e-05 3.07758512e-08
  1.21395822e-14 3.23247373e-01 5.46436611e-11 1.74970396e-07
  1.42703974e-03 1.18444077e-09 4.38608788e-03 7.40460928e-06
  2.80231878e-04 6.57436540e-05 9.99867638e-08 1.34895668e-08
  4.45239268e-09 3.21951939e-06 2.62458826e-07 1.73998116e-09
  2.58216751e-04 1.10512155e-10 3.33645089e-09 1.63408087e-09
  6.37020926e-13]]


In [61]:
predicted_index = np.argmax(predict_array)

print(predicted_index)

9


In [62]:
df = pd.read_csv('bird_sightings.csv')

df.head()

,Unnamed: 0,Recording_ID,English_name,Species,Country,Spectrogram_med,Date,Url,Latitude,Longitude,Vocalization_type,Recordist,mp3_url
0,43,234032,Common Chaffinch,Fringilla coelebs,United Kingdom,//www.xeno-canto.org/sounds/uploaded/BTOFEKXFG...,3/15/2015,//www.xeno-canto.org/234032,54.1246,-0.5468,"call, male, song",david m,//www.xeno-canto.org/sounds/uploaded/BTOFEKXFG...
1,44,168175,Common Chaffinch,Fringilla coelebs,United Kingdom,//www.xeno-canto.org/sounds/uploaded/BTOFEKXFG...,3/30/2012,//www.xeno-canto.org/168175,54.1248,-0.5719,"male, song",david m,//www.xeno-canto.org/sounds/uploaded/BTOFEKXFG...
2,45,213928,Common Chaffinch,Fringilla coelebs,United Kingdom,//www.xeno-canto.org/sounds/uploaded/BTOFEKXFG...,2/15/2015,//www.xeno-canto.org/213928,54.1246,-0.5702,"male, song",david m,//www.xeno-canto.org/sounds/uploaded/BTOFEKXFG...
3,46,512407,Common Chaffinch,Fringilla coelebs,Poland,//www.xeno-canto.org/sounds/uploaded/ZNCDXTUOF...,6/22/2015,//www.xeno-canto.org/512407,52.3724,20.6421,"adult, male, song",Jarek Matusiak,//www.xeno-canto.org/sounds/uploaded/ZNCDXTUOF...
4,47,177404,Common Chaffinch,Fringilla coelebs,United Kingdom,//www.xeno-canto.org/sounds/uploaded/BTOFEKXFG...,3/8/2014,//www.xeno-canto.org/177404,54.1288,-0.5471,"call, male, song",david m,//www.xeno-canto.org/sounds/uploaded/BTOFEKXFG...


In [63]:
species = []
common_name = []

for index, row in df.iterrows():
    sp = row['Species']
    cn = row['English_name']
    if sp not in species and cn not in common_name:
        species.append(sp)
        common_name.append(cn)
    else:
        pass

In [64]:
unique_df = pd.DataFrame({'Common Name': common_name, 'Species': species})
unique_df.head()

,Common Name,Species
0,Common Chaffinch,Fringilla coelebs
1,Great Tit,Parus major
2,Common Blackbird,Turdus merula
3,Song Thrush,Turdus philomelos
4,Common Whitethroat,Sylvia communis


In [65]:
new_df = unique_df.sort_values('Species')
new_df.head()

,Common Name,Species
16,Great Reed Warbler,Acrocephalus arundinaceus
17,Blyth's Reed Warbler,Acrocephalus dumetorum
40,Marsh Warbler,Acrocephalus palustris
39,Sedge Warbler,Acrocephalus schoenobaenus
27,Boreal Owl,Aegolius funereus


In [66]:
new_df = new_df.reset_index(drop = True)

new_df.head(20)

,Common Name,Species
0,Great Reed Warbler,Acrocephalus arundinaceus
1,Blyth's Reed Warbler,Acrocephalus dumetorum
2,Marsh Warbler,Acrocephalus palustris
3,Sedge Warbler,Acrocephalus schoenobaenus
4,Boreal Owl,Aegolius funereus
5,Eurasian Skylark,Alauda arvensis
6,Little Owl,Athene noctua
7,Eurasian Eagle-Owl,Bubo bubo
8,European Nightjar,Caprimulgus europaeus
9,European Goldfinch,Carduelis carduelis


In [68]:
new_df.to_csv('unique_species.csv')

In [74]:
index_df = pd.read_csv('unique_species.csv')

index_df = index_df[['Common Name', 'Species']]

index_df.head(20)

,Common Name,Species
0,Great Reed Warbler,Acrocephalus arundinaceus
1,Blyth's Reed Warbler,Acrocephalus dumetorum
2,Marsh Warbler,Acrocephalus palustris
3,Sedge Warbler,Acrocephalus schoenobaenus
4,Boreal Owl,Aegolius funereus
5,Eurasian Skylark,Alauda arvensis
6,Little Owl,Athene noctua
7,Eurasian Eagle-Owl,Bubo bubo
8,European Nightjar,Caprimulgus europaeus
9,European Goldfinch,Carduelis carduelis


In [73]:
for index, rows in index_df.iterrows():
    if index == predicted_index:
        sp = index_df.loc[index, 'Common Name']
        cn = index_df.loc[index, 'Species']
        print(sp, cn)

European Goldfinch Carduelis carduelis


In [87]:
def upload_file(file):
    name, ext = os.path.splitext(file)
    sound = AudioSegment.from_mp3(file)
    # make 30s clip 
    if len(sound) < 30000:
        duration = 30000-len(sound)
        segment = AudioSegment.silent(duration = duration)
        extract =  sound + segment
    else:
        extract = sound[0:30000]
    
    export = extract.export(f"{name}.wav", format="wav")
    
    new_file = f"{name}.wav"
            
    signal, sr = librosa.load(new_file, sr = sample_rate)

    mfcc_list = []

    for s in range(num_segments):
        start_sample = num_samples_per_segment * s
        finish_sample = start_sample + num_samples_per_segment
        mfccs = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                    sr = sample_rate, n_fft = n_fft, 
                                    n_mfcc = n_mfcc, hop_length = hop_length)
        mfccs = mfccs.T
        if len(mfccs) == expected_vectors_per_segment: 
            mfcc_list.append(np.array(mfccs))

        
    data = np.array(mfcc_list)
    X = data[..., np.newaxis]
    
    
    bird_model = load_model('cnn_66_trained.h5')
    prediction_array = bird_model.predict(X)

    predicted_index = np.argmax(prediction_array, axis=1)

    species_df = pd.read_csv('unique_species.csv')
    
    for index, rows in species_df.iterrows():
        if index == predicted_index:
            spec = species_df.loc[index, 'Common Name']
            cname = species_df.loc[index, 'Species']
            prediction = (spec, cname)

    
    return prediction

In [88]:
upload_file(file)

('European Goldfinch', 'Carduelis carduelis')